In [1]:
%%time

from seagul.rl.algos import sac, ppo
#from seagul.rl.algos.sac_ray import ray_sac
from seagul.rl.algos.sac_sym import sac_sym
from seagul.nn import MLP
from seagul.rl.models import SACModel, PPOModel 

input_size = 3
output_size = 1
layer_size = 64
num_layers = 2

policy = MLP(input_size, output_size*2, num_layers, layer_size)
value_fn = MLP(input_size, 1, num_layers, layer_size)
q1_fn = MLP(input_size + output_size, 1, num_layers, layer_size)
q2_fn = MLP(input_size + output_size, 1, num_layers, layer_size)
model = SACModel(policy, value_fn, q1_fn, q2_fn, 3)


ppo_policy = MLP(input_size, output_size, num_layers, layer_size)
ppo_model = PPOModel(ppo_policy, value_fn)

env_name = "Pendulum-v0"
#model, rews, var_dict = ray_sac(env_name, 20000, model, env_steps=0, iters_per_update=100, min_steps_per_update=100, reward_stop=-200, exploration_steps=100)
#model, rews, var_dict = ppo(env_name, 3e5, ppo_model)

for seed in [0,1,2,3]:
    %time model, rews, var_dict = sac_sym(env_name, 20000, model, seed=seed, env_steps=0, iters_per_update=100, min_steps_per_update=100, reward_stop=-200, exploration_steps=100)


for seed in [0,1,2,3]:
    %time model, rews, var_dict = sac(env_name, 20000, model, seed=seed, env_steps=0, iters_per_update=100, min_steps_per_update=100, reward_stop=-200, exploration_steps=100)


globals().update(var_dict)

/Users/sgillen/work/seagul/seagul/rl/algos/__init__.py:9: UserWarning: tensorflow not installed, skipping symmetric ppo
  warnings.warn("tensorflow not installed, skipping symmetric ppo")
/Users/sgillen/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)
  0%|          | 0/20000 [00:00<?, ?it/s]

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


  0%|          | 0/20000 [00:00<?, ?it/s]

CPU times: user 2min 39s, sys: 15.3 s, total: 2min 54s
Wall time: 1min
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.



 24%|██▍       | 4800/20000 [00:58<03:24, 74.28it/s]

  0%|          | 0/20000 [00:00<?, ?it/s]

CPU times: user 2min 42s, sys: 16.9 s, total: 2min 59s
Wall time: 1min 3s
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.




 24%|██▍       | 4800/20000 [01:10<03:24, 74.28it/s]

  6%|▌         | 1200/20000 [00:06<01:34, 199.67it/s]

  8%|▊         | 1600/20000 [00:11<02:17, 133.95it/s]

 10%|█         | 2000/20000 [00:17<02:46, 108.20it/s]


  0%|          | 0/20000 [00:00<?, ?it/s]

CPU times: user 58.1 s, sys: 5.87 s, total: 1min 3s
Wall time: 22.4 s
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.





  4%|▍         | 800/20000 [00:00<00:06, 3046.38it/s]


  6%|▌         | 1200/20000 [00:05<01:18, 238.71it/s]


  8%|▊         | 1600/20000 [00:10<02:06, 145.33it/s]

 10%|█         | 2000/20000 [00:36<02:46, 108.20it/s]

CPU times: user 42.8 s, sys: 4.06 s, total: 46.9 s
Wall time: 15.9 s
CPU times: user 7min 3s, sys: 42.3 s, total: 7min 45s
Wall time: 2min 42s


In [ ]:
import matplotlib.pyplot as plt

plt.plot(raw_rew_hist)
plt.figure()
plt.plot(pol_loss_hist)
plt.figure()
plt.plot(val_loss_hist)
plt.figure()
plt.plot(q1_loss_hist)
plt.plot(q2_loss_hist)


In [ ]:
replay_obs1, replay_obs2, replay_acts, replay_rews, replay_done = replay_buf.sample_batch(replay_batch_size)


In [ ]:
replay_buf.sample_batch()

In [ ]:
import gym
import torch
env = gym.make(env_name)
done = False
obs = env.reset()

obs_list = []
act_list = []
rew_list = []
avg_list = []
val_list = []

 
dtype = torch.float32
act_size = env.action_space.shape[0]
obs = env.reset()
done = False

while not done:
    model_obs = torch.as_tensor(obs, dtype=dtype).detach() #TODO.... (or not, this still works with ppo right??)
    obs_list.append(obs)
    
    noise = torch.randn(act_size)
    act, logp = model.select_action(model_obs.reshape(1, -1), noise)
    act = act.detach()

    obs, rew, done, _ = env.step(act.numpy().reshape(-1))
    env.render()

    act_list.append(act)
    rew_list.append(rew)

print(sum(rew_list))
plt.plot(obs_list)
plt.figure()
plt.plot(act_list)
plt.figure()
plt.plot(rew_list)
plt.figure()
plt.plot(val_list)
plt.figure()
plt.plot(avg_list)
    